In [ ]:
#extracting gender data
import tarfile

# Define the path to your tar file
tar_file_path = "/content/cv-corpus-21.0-delta-2025-03-14-en.tar.gz" # or "your_archive.tar.gz" for gzipped tar files
# Define the destination directory where contents will be extracted
destination_directory = "extracted_files"

try:
    # Open the tar file in read mode ('r' for .tar, 'r:gz' for .tar.gz)
    with tarfile.open(tar_file_path, "r") as tar:
        # Extract all contents to the specified directory
        tar.extractall(path=destination_directory)
    print(f"Successfully extracted '{tar_file_path}' to '{destination_directory}'.")
except FileNotFoundError:
    print(f"Error: The file '{tar_file_path}' was not found.")
except tarfile.ReadError:
    print(f"Error: Could not read the tar file '{tar_file_path}'. It might be corrupted or not a valid tar archive.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Successfully extracted '/content/cv-corpus-21.0-delta-2025-03-14-en.tar.gz' to 'extracted_files'.


In [ ]:
pip install pandas pydub soundfile

In [ ]:
import pandas as pd

df = pd.read_csv("/content/extracted_files/cv-corpus-21.0-delta-2025-03-14/en/validated.tsv", sep="\t")
##print(df.columns)
print(df[['path', 'gender']].dropna().head(10))
print("Unique genders:", df['gender'].dropna().unique())
count= df.groupby('gender').size()
print(count)
df.info()

                            path           gender
16  common_voice_en_42251480.mp3   male_masculine
18  common_voice_en_42594358.mp3  female_feminine
21  common_voice_en_42216083.mp3   male_masculine
22  common_voice_en_42466107.mp3  female_feminine
25  common_voice_en_42511436.mp3  female_feminine
26  common_voice_en_42511438.mp3  female_feminine
37  common_voice_en_41969910.mp3  female_feminine
38  common_voice_en_42182410.mp3  female_feminine
39  common_voice_en_42264420.mp3  female_feminine
40  common_voice_en_41918312.mp3   male_masculine
Unique genders: ['male_masculine' 'female_feminine' 'do_not_wish_to_say']
gender
do_not_wish_to_say      9
female_feminine       143
male_masculine         42
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 249 entries, 0 to 248
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   client_id        249 non-null    object 
 1   path             249 non-null 

In [ ]:
female_df = df[df['gender'] == 'female_feminine']
male_df = df[df['gender'] == 'male_masculine']
missing_count = 0
missing_counts=0
for fname in female_df['path']:
    if not os.path.exists(os.path.join("/content/extracted_files/cv-corpus-21.0-delta-2025-03-14/en/clips", fname)):
        missing_count += 1

for fname in male_df['path']:
    if not os.path.exists(os.path.join("/content/extracted_files/cv-corpus-21.0-delta-2025-03-14/en/clips", fname)):
        missing_counts += 1
print(f"Female entries in TSV: {len(female_df)}")
print(f"male entries in TSV: {len(male_df)}")
print(f"Missing female audio files: {missing_count}")
print(f"Missing male audio files: {missing_counts}")

Female entries in TSV: 143
male entries in TSV: 42
Missing female audio files: 0
Missing male audio files: 0


In [ ]:
import os
import pandas as pd
from pydub import AudioSegment
from tqdm import tqdm

def prepare_common_voice_gender(common_voice_path="common_voice/en", target_folder="data/gender"):
    os.makedirs(target_folder, exist_ok=True)

    # Load TSV
    tsv_path = os.path.join(common_voice_path, "validated.tsv")
    df = pd.read_csv(tsv_path, sep="\t")
    clips_path = os.path.join(common_voice_path, "clips")
    count = {"male": 0, "female": 0}
    missing_files = 0

    for idx, row in tqdm(df.iterrows(), total=len(df)):
        gender = row.get("gender")
        filename = row.get("path")

        if pd.isna(gender) or pd.isna(filename):
          continue

        if gender == "male_masculine":
           gender_clean = "male"
        elif gender == "female_feminine":
           gender_clean = "female"
        else:
           continue

        src_path = os.path.join(clips_path, filename)
        if not os.path.exists(src_path):
            missing_files += 1
            continue

        try:
            # Convert to WAV
            audio = AudioSegment.from_file(src_path)
            out_filename = f"{gender_clean}_{count[gender_clean]}.wav"
            out_path = os.path.join(target_folder, out_filename)
            audio.export(out_path, format="wav")
            count[gender_clean] += 1
        except Exception as e:
            print(f"⚠️ Error processing {filename}: {e}")

    print(f"\n✅ Done! {count['male']} male & {count['female']} female files saved.")
    if missing_files:
        print(f"⚠️ Skipped {missing_files} files that were missing in /clips.")

# Run it
prepare_common_voice_gender("/content/extracted_files/cv-corpus-21.0-delta-2025-03-14/en", "data/gender")


100%|██████████| 249/249 [00:37<00:00,  6.65it/s]


✅ Done! 42 male & 143 female files saved.


In [ ]:
#using the same dataset for age data
import os
import pandas as pd
from pydub import AudioSegment
from tqdm import tqdm

def prepare_common_voice_age(common_voice_path="common_voice/en", target_root="data/age"):
    os.makedirs(target_root, exist_ok=True)

    df = pd.read_csv(os.path.join(common_voice_path, "validated.tsv"), sep="\t")
    clips_path = os.path.join(common_voice_path, "clips")

    valid_ages = ["teens", "twenties", "thirties", "forties", "fifties", "sixties", "seventies", "eighties"]
    count = {age: 0 for age in valid_ages}
    missing_files = 0

    for _, row in tqdm(df.iterrows(), total=len(df)):
        gender = row.get("gender")
        age = row.get("age")
        filename = row.get("path")

        if pd.isna(gender) or pd.isna(age) or pd.isna(filename):
            continue
        if gender != "male_masculine" or age not in valid_ages:
            continue

        src_path = os.path.join(clips_path, filename)
        if not os.path.exists(src_path):
            missing_files += 1
            continue

        try:
            age_folder = os.path.join(target_root, age)
            os.makedirs(age_folder, exist_ok=True)

            audio = AudioSegment.from_file(src_path)
            out_filename = f"{age}_{count[age]}.wav"
            audio.export(os.path.join(age_folder, out_filename), format="wav")
            count[age] += 1
        except Exception as e:
            print(f"⚠️ Error processing {filename}: {e}")

    print(f"\n✅ Done! Files per age group: {count}")
    if missing_files:
        print(f"⚠️ Skipped {missing_files} missing files.")

# Run this to prepare your dataset
prepare_common_voice_age("/content/extracted_files/cv-corpus-21.0-delta-2025-03-14/en", "data/age")



100%|██████████| 249/249 [00:07<00:00, 31.62it/s]


✅ Done! Files per age group: {'teens': 0, 'twenties': 3, 'thirties': 37, 'forties': 0, 'fifties': 0, 'sixties': 1, 'seventies': 0, 'eighties': 0}


In [ ]:
import os
import shutil

def convert_to_binary_age_class(src_path="data/age", dest_path="data/age_binary"):
    senior_groups = ["sixties", "seventies", "eighties"]
    non_senior_groups = ["teens", "twenties", "thirties", "forties", "fifties"]

    for age_group in os.listdir(src_path):
        group_path = os.path.join(src_path, age_group)
        if not os.path.isdir(group_path):
            continue

        label = "senior" if age_group in senior_groups else "non_senior"
        out_folder = os.path.join(dest_path, label)
        os.makedirs(out_folder, exist_ok=True)

        for file in os.listdir(group_path):
            src_file = os.path.join(group_path, file)
            shutil.copy(src_file, os.path.join(out_folder, file))

    print("✅ Age data converted to binary classification format.")

# Run this once
convert_to_binary_age_class()


✅ Age data converted to binary classification format.


In [ ]:
#Augmenting the only data file we had into 10 more files for model
import librosa
import soundfile as sf
import os

def augment_audio(file_path, out_dir, count=10):
    y, sr = librosa.load(file_path, sr=None)
    os.makedirs(out_dir, exist_ok=True)

    for i in range(count):
        n_steps = (i % 5) - 2  # values: -2, -1, 0, 1, 2
        y_shifted = librosa.effects.pitch_shift(y, sr=sr, n_steps=n_steps)
        out_file = os.path.join(out_dir, f"senior_aug_{i}.wav")
        sf.write(out_file, y_shifted, sr)
        print(f"Saved: {out_file}")

# Run this with your actual sixties file path
augment_audio("data/age/sixties/sixties_0.wav", "data/age_binary/senior", count=10)


Saved: data/age_binary/senior/senior_aug_0.wav
Saved: data/age_binary/senior/senior_aug_1.wav
Saved: data/age_binary/senior/senior_aug_2.wav
Saved: data/age_binary/senior/senior_aug_3.wav
Saved: data/age_binary/senior/senior_aug_4.wav
Saved: data/age_binary/senior/senior_aug_5.wav
Saved: data/age_binary/senior/senior_aug_6.wav
Saved: data/age_binary/senior/senior_aug_7.wav
Saved: data/age_binary/senior/senior_aug_8.wav
Saved: data/age_binary/senior/senior_aug_9.wav


In [1]:
#emotion dataset
!unzip '/content/archive (2).zip'

Archive:  /content/archive (2).zip
  inflating: Actor_01/03-01-01-01-01-01-01.wav  
  inflating: Actor_01/03-01-01-01-01-02-01.wav  
  inflating: Actor_01/03-01-01-01-02-01-01.wav  
  inflating: Actor_01/03-01-01-01-02-02-01.wav  
  inflating: Actor_01/03-01-02-01-01-01-01.wav  
  inflating: Actor_01/03-01-02-01-01-02-01.wav  
  inflating: Actor_01/03-01-02-01-02-01-01.wav  
  inflating: Actor_01/03-01-02-01-02-02-01.wav  
  inflating: Actor_01/03-01-02-02-01-01-01.wav  
  inflating: Actor_01/03-01-02-02-01-02-01.wav  
  inflating: Actor_01/03-01-02-02-02-01-01.wav  
  inflating: Actor_01/03-01-02-02-02-02-01.wav  
  inflating: Actor_01/03-01-03-01-01-01-01.wav  
  inflating: Actor_01/03-01-03-01-01-02-01.wav  
  inflating: Actor_01/03-01-03-01-02-01-01.wav  
  inflating: Actor_01/03-01-03-01-02-02-01.wav  
  inflating: Actor_01/03-01-03-02-01-01-01.wav  
  inflating: Actor_01/03-01-03-02-01-02-01.wav  
  inflating: Actor_01/03-01-03-02-02-01-01.wav  
  inflating: Actor_01/03-01-03-02-

In [2]:
#gender detection model training
import os
import numpy as np
import librosa
import tensorflow as tf
from sklearn.model_selection import train_test_split

# === Dataset Path ===
dataset_path = "/content/audio_speech_actors_01-24"

def extract_mfcc(file_path, max_len=160):
    y, sr = librosa.load(file_path, sr=22050)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    if mfcc.shape[1] < max_len:
        pad_width = max_len - mfcc.shape[1]
        mfcc = np.pad(mfcc, pad_width=((0, 0), (0, pad_width)), mode='constant')
    else:
        mfcc = mfcc[:, :max_len]
    return mfcc.T  # shape: (160, 13)

# === Prepare Data ===
X, y = [], []

for actor_dir in sorted(os.listdir(dataset_path)):
    actor_path = os.path.join(dataset_path, actor_dir)
    if not actor_dir.startswith("Actor_"):
        continue

    actor_id = int(actor_dir.split("_")[1])
    gender_label = 1 if actor_id % 2 == 1 else 0  # 1 = male, 0 = female

    for file in os.listdir(actor_path):
        if file.endswith(".wav"):
            file_path = os.path.join(actor_path, file)
            try:
                mfcc = extract_mfcc(file_path)
                X.append(mfcc)
                y.append(gender_label)
            except Exception as e:
                print(f"[SKIPPED] {file_path} due to {e}")

X = np.array(X)
y = np.array(y)

print(f"✅ Dataset prepared: {X.shape}, Labels: {y.shape}")

# === Train/Test Split ===
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# === Model Architecture ===
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(160, 13)),
    tf.keras.layers.Conv1D(64, 3, activation='relu'),
    tf.keras.layers.MaxPooling1D(2),
    tf.keras.layers.Conv1D(128, 3, activation='relu'),
    tf.keras.layers.MaxPooling1D(2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

# === Train ===
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20, batch_size=32)

# === Save the Model ===
model.save("gender_model.keras")
print("Saved retrained gender model as gender_model.keras")


✅ Dataset prepared: (1440, 160, 13), Labels: (1440,)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 158, 64)        │         2,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 79, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 77, 128)        │        24,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 38, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 4864)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │       311,360 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 338,689 (1.29 MB)

 Trainable params: 338,689 (1.29 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 5s 17ms/step - accuracy: 0.5325 - loss: 24.6948 - val_accuracy: 0.7222 - val_loss: 0.6108
Epoch 2/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7591 - loss: 0.5513 - val_accuracy: 0.8924 - val_loss: 0.4164
Epoch 3/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8756 - loss: 0.4612 - val_accuracy: 0.9306 - val_loss: 0.3909
Epoch 4/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9022 - loss: 0.4019 - val_accuracy: 0.9653 - val_loss: 0.3661
Epoch 5/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9305 - loss: 0.3973 - val_accuracy: 0.9618 - val_loss: 0.3720
Epoch 6/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9282 - loss: 0.3824 - val_accuracy: 0.9722 - val_loss: 0.3497
Epoch 7/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9471 - loss: 0.3648 - val_accuracy: 0.9549 - val_loss: 0.3374
Epoch 8/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9504 - loss: 0.3410 - val_accuracy: 0.9861 - val_los

In [ ]:
#age detection model
import os
import numpy as np
import librosa
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

# --- Step 1: Extract MFCC features ---
def extract_features(file_path, n_mfcc=13):
    y, sr = librosa.load(file_path, sr=None)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
    return np.mean(mfcc.T, axis=0)

# --- Step 2: Prepare dataset ---
def prepare_age_data(data_path="data/age_binary"):
    X, y = [], []
    for label in os.listdir(data_path):
        folder = os.path.join(data_path, label)
        if not os.path.isdir(folder):
            continue
        for file in os.listdir(folder):
            if not file.endswith(".wav"):
                continue
            file_path = os.path.join(folder, file)
            try:
                features = extract_features(file_path)
                X.append(features)
                y.append(label)
            except Exception as e:
                print(f"Error processing {file_path}: {e}")
    return np.array(X), np.array(y)

# --- Step 3: Load and encode data ---
X, y = prepare_age_data("data/age_binary")
print("Loaded data:", X.shape, y.shape)

le = LabelEncoder()
y_encoded = le.fit_transform(y)  # senior = 1, non_senior = 0

# Split with stratification
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.25, stratify=y_encoded, random_state=42
)

# --- Step 4: Build model ---
model = Sequential([
    Dense(64, activation='relu', input_shape=(X.shape[1],)),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')  # Binary output
])

model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

# --- Step 5: Train model ---
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=8)

# --- Step 6: Evaluate and Save ---
loss, acc = model.evaluate(X_test, y_test)
print(f"✅ Age Binary Model Accuracy: {acc:.2f}")

model.save("age_binary_model.keras")


Loaded data: (51, 13) (51,)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_9 (Dense)                 │ (None, 64)             │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,009 (11.75 KB)

 Trainable params: 3,009 (11.75 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 694ms/step - accuracy: 0.6239 - loss: 25.9256 - val_accuracy: 0.7692 - val_loss: 8.9825
Epoch 2/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.5786 - loss: 14.8120 - val_accuracy: 0.7692 - val_loss: 8.7376
Epoch 3/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.7366 - loss: 12.5233 - val_accuracy: 0.7692 - val_loss: 7.6920
Epoch 4/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.5993 - loss: 13.4952 - val_accuracy: 0.7692 - val_loss: 6.5221
Epoch 5/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.6308 - loss: 11.1390 - val_accuracy: 0.7692 - val_loss: 5.5284
Epoch 6/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.6602 - loss: 9.6685 - val_accuracy: 0.7692 - val_loss: 4.4534
Epoch 7/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.6969 - loss: 10.1794 - val_accuracy: 0.7692 - val_loss: 3.0823
Epoch 8/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.7316 - loss: 5.6690 - val_accuracy: 0.7692 - val_loss: 

In [ ]:
#As there were only 720 data samples intially the model gave an accuracy of near 24%. Therefore to increase the accuracy augmentation
#was performed which increased the accuracy significantly
import os
import numpy as np
import librosa
import pickle
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

# === Paths ===
data_dir = "/content/audio_speech_actors_01-24"
save_dir = "data/emotion"
os.makedirs(save_dir, exist_ok=True)

# === Emotion code mapping ===
emotion_map = {
    '01': 'neutral',
    '02': 'calm',
    '03': 'happy',
    '04': 'sad',
    '05': 'angry',
    '06': 'fearful',
    '07': 'disgust',
    '08': 'surprised'
}

# === Feature extraction from audio array ===
def extract_features_from_array(y, sr, max_len=160):
    try:
        if len(y) < sr * 0.5:  # skip very short clips
            return None

        mel = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128)
        if mel.shape[1] == 0:
            return None
        mel_db = librosa.power_to_db(mel, ref=np.max)

        if mel_db.shape[1] < max_len:
            pad_width = max_len - mel_db.shape[1]
            mel_db = np.pad(mel_db, pad_width=((0, 0), (0, pad_width)), mode='constant')
        else:
            mel_db = mel_db[:, :max_len]

        return mel_db
    except Exception as e:
        print(f"[AUG ERROR] Feature extraction failed: {e}")
        return None

# === Audio augmentation function ===
def augment_audio(y, sr):
    try:
        y_aug = librosa.effects.pitch_shift(y=y, sr=sr, n_steps=np.random.uniform(-2, 2))
        noise = np.random.normal(0, 0.005, y_aug.shape)
        y_aug = y_aug + noise
        return y_aug
    except Exception as e:
        print(f"[AUG ERROR] Augmentation failed: {e}")
        return None

# === Data extraction loop ===
X, y = [], []
skipped = 0

for root, dirs, files in os.walk(data_dir):
    for file in files:
        if file.endswith('.wav'):
            parts = file.split("-")
            emotion_code = parts[2]
            actor_id = int(parts[-1].split(".")[0])

            # Only male actors (odd-numbered IDs)
            if actor_id % 2 == 1 and emotion_code in emotion_map:
                file_path = os.path.join(root, file)

                try:
                    # Load original audio
                    y_raw, sr = librosa.load(file_path, sr=22050)

                    # --- Original ---
                    feat_orig = extract_features_from_array(y_raw, sr)
                    if feat_orig is not None:
                        X.append(feat_orig)
                        y.append(emotion_map[emotion_code])
                    else:
                        print(f"[SKIP ORIG] {file_path}")
                        skipped += 1

                    # --- Augmented ---
                    y_aug = augment_audio(y_raw, sr)
                    if y_aug is not None:
                        feat_aug = extract_features_from_array(y_aug, sr)
                        if feat_aug is not None:
                            X.append(feat_aug)
                            y.append(emotion_map[emotion_code])
                        else:
                            print(f"[SKIP AUG FEAT] {file_path}")
                            skipped += 1
                    else:
                        print(f"[SKIP AUG AUDIO] {file_path}")
                        skipped += 1

                except Exception as e:
                    print(f"[SKIP ERROR] {file_path}: {e}")
                    skipped += 1

print(f"✅ Total samples extracted: {len(X)} | Skipped: {skipped}")

# === Save dataset ===
X = np.array(X)
X = X[..., np.newaxis]

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
y_cat = to_categorical(y_encoded)

np.save(os.path.join(save_dir, "X_aug.npy"), X)
np.save(os.path.join(save_dir, "y_aug.npy"), y_cat)

with open(os.path.join(save_dir, "label_encoder.pkl"), "wb") as f:
    pickle.dump(label_encoder, f)

print(f"✅ Saved augmented dataset to '{save_dir}'")
print(f"    ➤ X shape: {X.shape}")
print(f"    ➤ y shape: {y_cat.shape}")


✅ Total samples extracted: 1440 | Skipped: 0
✅ Saved augmented dataset to 'data/emotion'
    ➤ X shape: (1440, 128, 160, 1)
    ➤ y shape: (1440, 8)


In [ ]:
#emotion model training
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Dropout, Flatten, Dense

# Load dataset
data_dir = "data/emotion"
X = np.load(f"{data_dir}/X_aug.npy")
y = np.load(f"{data_dir}/y_aug.npy")

with open(f"{data_dir}/label_encoder.pkl", "rb") as f:
    label_encoder = pickle.load(f)

# Class weights
y_int = np.argmax(y, axis=1)
class_weights = compute_class_weight('balanced', classes=np.unique(y_int), y=y_int)
class_weights = dict(enumerate(class_weights))

# Train/val split
X_train, X_val, y_train, y_val = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

# Model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 160, 1)),
    MaxPooling2D((2, 2)),
    BatchNormalization(),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    BatchNormalization(),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    BatchNormalization(),

    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(y.shape[1], activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=50,
    batch_size=32,
    class_weight=class_weights,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)]
)

# Save model
model.save("/content/emotion_model.keras")
print("✅ Emotion model saved as /content/emotion_model.keras")


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 10s 71ms/step - accuracy: 0.1932 - loss: 9.0853 - val_accuracy: 0.1632 - val_loss: 18.1259
Epoch 2/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 4s 29ms/step - accuracy: 0.2040 - loss: 2.0820 - val_accuracy: 0.1910 - val_loss: 2.9423
Epoch 3/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - accuracy: 0.2412 - loss: 1.9340 - val_accuracy: 0.1944 - val_loss: 2.2151
Epoch 4/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - accuracy: 0.2512 - loss: 1.9593 - val_accuracy: 0.3021 - val_loss: 1.9103
Epoch 5/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 0.2719 - loss: 1.9193 - val_accuracy: 0.2882 - val_loss: 1.9525
Epoch 6/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 0.2851 - loss: 1.8730 - val_accuracy: 0.2882 - val_loss: 1.9903
Epoch 7/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - accuracy: 0.2924 - loss: 1.8032 - val_accuracy: 0.3160 - val_loss: 1.8371
Epoch 8/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - accuracy: 0.2858 - loss: 1.8643 - val_accuracy: 0.3403 -

In [ ]:
gender_model.summary()


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_6 (Dense)                 │ (None, 64)             │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 9,029 (35.27 KB)

 Trainable params: 3,009 (11.75 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 6,020 (23.52 KB)